# Imports

In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import json
import pycountry

# API Testing

In [32]:
r = requests.get(
    "https://www.vivino.com/api/explore/explore",
    params = {
        "country_code": "FR",
        "country_codes[]":"pt",
        "currency_code":"EUR",
        "grape_filter":"varietal",
        "min_rating":"1",
        "order_by":"price",
        "order":"asc",
        "page": 1,
        "price_range_max":"500",
        "price_range_min":"0",
        "wine_type_ids[]":"1"
    },
    headers= {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:66.0) Gecko/20100101 Firefox/66.0"
    }
)
results = [
    (
        t["vintage"]["wine"]["winery"]["name"], 
        f'{t["vintage"]["wine"]["name"]} {t["vintage"]["year"]}',
        t["vintage"]["statistics"]["ratings_average"],
        t["vintage"]["statistics"]["ratings_count"],
    )
    for t in r.json()["explore_vintage"]["matches"]
]
dataframe = pd.DataFrame(results,columns=['Winery','Wine','Rating','num_review'])

print(dataframe)


                     Winery                                  Wine  Rating  \
0                  Graça 28                          Reserva 2022     4.1   
1                 Vila Real                    Vale do Corgo 2021     3.8   
2    Casa Ermelinda Freitas      Sandstone Castelão - Shiraz 2023     4.0   
3                   Cartuxa                         EA Tinto 2021     3.8   
4   Quinta de São Sebastião         Miradouro da Vinha Tinto 2022     3.8   
5   Quinta de São Sebastião         Miradouro da Vinha Tinto 2021     3.8   
6         Casal das Freiras                   Colheita Tinto 2022     3.5   
7   Quinta de São Sebastião  Janela Branca Special Selection 2022     3.7   
8                 Vila Real                Cancellus Reserva 2017     3.8   
9       Quinta Dona Mafalda                   Little Mafalda 2022     3.8   
10            Monte do Além                     Petit Verdot 2012     3.8   
11                Vila Real              Cancellus Signature 2019     3.9   

# Vivino API Call

In [2]:
# Define function

def get_vivino_data(
    name=None,
    country_code=None,
    currency_code=None,
    min_rating=0,
    ratings_count_min=0,
    price_range_max=100000,
    price_range_min=0,
    wine_type_ids=1,
):
    r = requests.get(
        "https://www.vivino.com/api/explore/explore",
        params={
            "search_query": name,
            "country_code": country_code,
            "currency_code": currency_code,
            "min_rating": min_rating,
            "min_ratings": ratings_count_min,
            "order_by": "price",
            "order": "asc",
            "page": 1,
            "price_range_max": price_range_max,
            "price_range_min": price_range_min,
            "wine_type_ids[]": wine_type_ids,
        },
        headers={
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:66.0) Gecko/20100101 Firefox/66.0"
        },
    )


    results = [
        {
            "name": t["vintage"]["wine"]["winery"]["name"],
            "id": t["vintage"]["id"],
            "wine": f'{t["vintage"]["wine"]["name"]} {t["vintage"]["year"]}',
            "region": t["vintage"]["wine"]["region"]["name"],
            "country": t["vintage"]["wine"]["region"]["country"]["name"],
            "rating": t["vintage"]["statistics"]["ratings_average"],
            "flavors": [x["group"] for x in t["vintage"]["wine"]["taste"]["flavor"]],
            "foods": [x["name"] for x in t["vintage"]["wine"]["style"]["food"]],
            "rating_count": t["vintage"]["statistics"]["ratings_count"],
        }
        for t in r.json()["explore_vintage"]["matches"]
    ]

    # Get lists
    wineries = [x["name"] for x in results]
    links = ["https://vivino.com/wines/" + str(x["id"]) for x in results] # Will expand to a bigger link when searched
    wines = [x["wine"] for x in results]
    ratings = [x["rating"] for x in results]
    flavors = [x["flavors"] for x in results]
    foods = [x["foods"] for x in results]
    country_code = [x["country"] for x in results]
    region = [x["region"] for x in results]

    # Clean flavors by removing _ and capitalizing
    flavors = [[y.replace("_", " ").title() for y in x] for x in flavors]

    # Get number of reviews
    num_reviews = [x["rating_count"] for x in results]

    return pd.DataFrame(
        {
            "winery": wineries,
            "wine": wines,
            "link": links,
            "region": region,
            "country": country_code,
            "rating": ratings,
            "flavors": flavors,
            "foods": foods,
            "num_reviews": num_reviews,
        }
    )


# Get data
dataframe = get_vivino_data(
    name="",
    country_code="FR",
    currency_code="USD",
    min_rating=1,
    ratings_count_min=1000,
    price_range_max=500,
    price_range_min=0,
    wine_type_ids=1,
)

dataframe.head()

,winery,wine,link,region,country,rating,flavors,foods,num_reviews
0,Château de Lavagnac,Bordeaux 2015,https://vivino.com/wines/93384618,Bordeaux,France,3.6,"[Red Fruit, Black Fruit, Earth, Oak, Non Oak, ...","[Beef, Veal, Game (deer, venison), Poultry]",76
1,Château de Callac,Graves Rouge 2011,https://vivino.com/wines/2137427,Graves,France,3.6,"[Earth, Oak, Black Fruit, Non Oak, Red Fruit, ...","[Beef, Lamb, Game (deer, venison), Poultry]",221
2,Château Lacoste Garzac,Bordeaux 2022,https://vivino.com/wines/171614480,Bordeaux,France,3.4,"[Oak, Black Fruit, Earth, Non Oak, Red Fruit, ...","[Beef, Veal, Game (deer, venison), Poultry]",2054
3,Château Haut Bernin,Bordeaux 2015,https://vivino.com/wines/146357367,Bordeaux,France,3.3,"[Black Fruit, Red Fruit, Non Oak, Oak, Vegetal...","[Beef, Veal, Game (deer, venison), Poultry]",69
4,Château La Fleur Saint-Jean,Bordeaux 2009,https://vivino.com/wines/1667151,Bordeaux,France,3.6,"[Red Fruit, Spices, Oak, Black Fruit, Microbio]","[Beef, Veal, Game (deer, venison), Poultry]",168


In [3]:
import PyPDF2

def extract_text_from_pdf(pdf_path):
    """
    Extract text from a PDF file.
    
    Args:
        pdf_path (str): Path to the PDF file
        
    Returns:
        dict: Dictionary containing page numbers and their corresponding text
    """
    # Dictionary to store text from each page
    text_by_page = {}
    
    try:
        # Open the PDF file in binary read mode
        with open(pdf_path, 'rb') as file:
            # Create a PDF reader object
            pdf_reader = PyPDF2.PdfReader(file)
            
            # Get the number of pages
            num_pages = len(pdf_reader.pages)
            
            # Extract text from each page
            for page_num in range(num_pages):
                # Get the page object
                page = pdf_reader.pages[page_num]
                
                # Extract text from the page
                text = page.extract_text()
                
                # Store the text in our dictionary
                text_by_page[page_num + 1] = text
                
        return text_by_page
    
    # Error Messaging
    except FileNotFoundError:
        print(f"Error: The file {pdf_path} was not found.")
        return None
    except PyPDF2.PdfReadError:
        print("Error: Invalid or corrupted PDF file.")
        return None
    except Exception as e:
        print(f"An unexpected error occurred: {str(e)}")
        return None



In [4]:
print(extract_text_from_pdf("menus/coucou-wine.pdf"))

{1: "SPARKLING  |  FRANCE  |  CHAMPAGNE\nNON-VINTAGE\n1148 Alexandre Bonnet / 'La Geande' 7 Cepages' / Extra Brut NV 310\n1147 Alexandre Filaine / 'Cuvée Spéciale' / Brut NV 235\n1159 Alexandre Filaine / 'DMY' / Brut NV 295\n1116 Georges Laval / 'Cumières' 1er cru / Brut Nature NV 288\n1199 Laurent Perrier / 'Grand Siècle - No. 29' / Brut NV 685\nBLANC DE BLANCS\n1100 Pascal Agrapart / 'Minèral' / Extra Brut 2018 350\n1187 Pascal Agrapart / 'Vénus' / Extra Brut 2018 650\n1124 Raphaël et Vincent Bérêche  / 'Côte' /  Grand Cru /  Extra Brut 2005 498\n1203 Billecart-Salmon / 'Cuvée Louis Salmon' / Brut 2008 585\n1146 Franck Bonville / 'Les Belles Voyes' / Brut 2016 298\n1115 Ulysse Collin / 'Les Roises' / Extra Brut 2011 1450\n1168 Ulysse Collin / 'Les Pierrières' /  Brut 2018 725\n1106 Ulysse Collin / 'Les Pierrières' /  Brut 2011 1095\n1144 Emilien Feneuil / 'Cuvée Totum' / Extra Brut chardonnay -  petit meslier 2018 495\n1178 Emilien Feneuil / 'Cuvée Mix' / Extra Brut chardonnay -  pet

# Create PDF Parser

In [36]:
from dotenv import load_dotenv
import os

load_dotenv(dotenv_path='config.env')
google_key = os.getenv('GOOGLE_KEY')

if google_key is None:
    raise ValueError("GOOGLE_KEY not found in .env file")

In [37]:
import google.generativeai as genai
from typing import List, Dict
import json
import PyPDF2

# Made with Claude 3.5

class GeminiWineParser:
    def __init__(self, api_key: str):
        """Initialize the Gemini parser with API key"""
        genai.configure(api_key=api_key)
        self.model = genai.GenerativeModel('gemini-1.5-pro')
        
    def parse_wine_list(self, text: str) -> List[Dict]:
        """
        Parse wine list text using Gemini 1.5
        
        Args:
            text (str): The wine list text to parse
            
        Returns:
            List[Dict]: List of parsed wine entries
        """
        prompt = f"""Extract wine information from the text below into a structured format.
        For each wine entry, extract:
        - ID number
        - Producer
        - Wine name
        - Type (e.g., NON-VINTAGE, BLANC DE BLANCS)
        - Main Type (e.g., SPARKLING, WHITE, RED, ROSE)
        - Region
        - Vintage (if available)
        - Price
        - Size (glass, bottle, half bottle, magnum)
        
        Format as JSON with missing fields as null but get as many wines as possible even if some fields are missing.
        
        Text to parse:
        {text}
        
        Respond with only valid JSON in this exact format:
        {{
            "wines": [
                {{
                    "id": "1234",
                    "producer": "Producer Name",
                    "name": "Wine Name",
                    "type": "Wine Type",
                    "region": "Region",
                    "country": "Country",
                    "vintage": "2020",
                    "price": "123",
                    "size": "bottle"
                }}
            ]
        }}"""

        try:
            response = self.model.generate_content(
                prompt,
                generation_config={
                    'temperature': 0.0,  # Use deterministic output
                    'top_p': 1.0,
                    'top_k': 1
                }
            )
            
            # Find the JSON in the response
            response_text = response.text
            # Look for JSON between ```json and ``` if present
            if '```json' in response_text:
                json_str = response_text.split('```json')[1].split('```')[0].strip()
            else:
                json_str = response_text.strip()
                
            # Parse the JSON response
            json_response = json.loads(json_str)
            return json_response["wines"]
            
        except Exception as e:
            print(f"Error parsing wine list: {str(e)}")
            return []

    def parse_pdf_and_wine_list(self, pdf_path: str, page_number: int = 1) -> List[Dict]:
        """
        Extract text from PDF and parse wine list
        
        Args:
            pdf_path (str): Path to PDF file
            page_number (int): Page number to parse (default: 1)
            
        Returns:
            List[Dict]: List of parsed wine entries
        """
        try:
            with open(pdf_path, 'rb') as file:
                reader = PyPDF2.PdfReader(file)
                if page_number <= len(reader.pages):
                    text = reader.pages[page_number - 1].extract_text()
                    return self.parse_wine_list(text)
                else:
                    raise ValueError(f"PDF has only {len(reader.pages)} pages")
        except Exception as e:
            print(f"Error processing PDF: {str(e)}")
            return []
    
    def save_to_json(self, wines: List[Dict], output_file: str):
        """Save parsed wines to a JSON file"""
        try:
            with open(output_file, 'w') as f:
                json.dump({'wines': wines}, f, indent=2)
            print(f"Successfully saved to {output_file}")
        except Exception as e:
            print(f"Error saving to file: {str(e)}")

# Add API Key
parser = GeminiWineParser(google_key)

# Example wine list text
sample_text = """SPARKLING  |  FRANCE  |  CHAMPAGNE
NON-VINTAGE
1148 Alexandre Bonnet / 'La Geande' 7 Cepages' / Extra Brut NV 310
1147 Alexandre Filaine / 'Cuvée Spéciale' / Brut NV 235
BLANC DE BLANCS
1100 Pascal Agrapart / 'Minèral' / Extra Brut 2018 350"""

# Parse text directly
results = parser.parse_wine_list(sample_text)

# Print results
print(json.dumps(results, indent=2))

[
  {
    "id": "1148",
    "producer": "Alexandre Bonnet",
    "name": "'La Geande' 7 Cepages'",
    "type": "Extra Brut",
    "main_type": "SPARKLING",
    "region": "CHAMPAGNE",
    "country": "FRANCE",
    "vintage": "NV",
    "price": "310",
    "size": "bottle"
  },
  {
    "id": "1147",
    "producer": "Alexandre Filaine",
    "name": "'Cuv\u00e9e Sp\u00e9ciale'",
    "type": "Brut",
    "main_type": "SPARKLING",
    "region": "CHAMPAGNE",
    "country": "FRANCE",
    "vintage": "NV",
    "price": "235",
    "size": "bottle"
  },
  {
    "id": "1100",
    "producer": "Pascal Agrapart",
    "name": "'Min\u00e8ral'",
    "type": "Extra Brut",
    "main_type": "SPARKLING",
    "region": "CHAMPAGNE",
    "country": "FRANCE",
    "vintage": "2018",
    "price": "350",
    "size": "bottle"
  }
]


In [38]:
text = extract_text_from_pdf("menus/rake-wine.pdf")
pages = [text[i] for i in text.keys()]

In [39]:
from tqdm import tqdm

parser = GeminiWineParser(google_key)


# Try with full page
for page in tqdm(pages[:1]):
    results = parser.parse_wine_list(page)
    
    
# Create dataframe
df = pd.DataFrame(results)
df

100%|██████████| 1/1 [00:35<00:00, 35.11s/it]


,id,producer,name,type,main_type,region,country,vintage,price,size
0,None,Chëpìka,Petnat Catawba,PETNAT,WHITE,Finger Lakes,USA,2021,64,bottle
1,None,La Ferme du Vert,L' Angelou Blanc Bulle,None,WHITE,Southwest,FRA,2021,56,bottle
2,None,Guiborat,Prisme - Blanc de Blancs,"BLANC DE BLANCS, GRAND CRU EXTRA BRUT",SPARKLING,Champagne,FRA,None,144,bottle
3,None,Pierre Moncuit,Delos Blanc de Blancs,"BLANC DE BLANCS, GRAND CRU BRUT",SPARKLING,Champagne,FRA,None,117,bottle
4,None,Robert Moncuit,Millésime Blanc de Blancs,"BLANC DE BLANCS, GRAND CRU BRUT",SPARKLING,Champagne,FRA,2013,196,bottle
5,None,Azienda Agricola Monban,Questo Neanche,COL FONDO,SPARKLING,Veneto,ITA,2021,45,bottle
6,None,Diletta Tonello,Marachelle,FRIZZANTE,WHITE,Veneto,ITA,None,53,bottle
7,None,Aldo Viola,Brutto,None,WHITE,Sicily,ITA,2022,49,bottle
8,None,Joao Pato,Ducking,PETNAT,WHITE,Bairrada,PRT,2022,61,bottle
9,None,Familie Bauer,Unsprung,PETNAT,WHITE,Wagram,AUT,2021,52,bottle


In [40]:
df.type.value_counts()

PETNAT                                   4
BLANC DE BLANCS, GRAND CRU BRUT          2
BLANC DE BLANCS, GRAND CRU EXTRA BRUT    1
COL FONDO                                1
FRIZZANTE                                1
FRIZZANTE ROSATO                         1
ROSÉ DE SAIGNÉE                          1
BRUT NATURE                              1
METODO TRADICIONAL                       1
METODE ANCESTRAL                         1
Name: type, dtype: int64

In [22]:
def get_vivino_table(wine_df):
    # Run get_vivino_data for each wine in the dataframe

    countries = wine_df["country"].unique()
    print(f"Getting data for {countries} countries")
    country_mapping = {country.name : country.alpha_2 for country in pycountry.countries if country.name in countries}

    wine_df["country_code"] = wine_df["country"].map(country_mapping)

    return wine_df

# Call the function
get_vivino_table(df)


Getting data for ['FRA' 'USA' 'DEU' 'SVN' 'CHL' 'ESP'] countries


,id,producer,name,type,main_type,region,country,vintage,price,country_code
0,None,Pierre Gimonnet & Fils,"""BdB"" 1er Cru Brut",BLANC DE BLANCS NV,SPARKLING,Champagne,FRA,None,68,NaN
1,None,Ployez-Jacquemart,Extra Brut Rosé,ROSÉ NV,SPARKLING,Champagne,FRA,None,62,NaN
2,None,Domaine Moreau-Naudet,Chablis,CHARDONNAY,WHITE,Burgundy,FRA,2020,52,NaN
3,None,Domaine Roger & Didier Raimbault,Sancerre,SAUVIGNON BLANC,WHITE,Loire,FRA,2020,33,NaN
4,None,Kistler,"""Les Noisetiers"", Sonoma Coast",CHARDONNAY,WHITE,California,USA,2020,71,NaN
5,None,Dashe Cellars,Dry Creek Valley Zinfandel,ZINFANDEL,RED,California,USA,2017,41,NaN
6,None,Corison,Napa Valley Cabernet Sauvignon,CABERNET SAUVIGNON,RED,California,USA,2018,116,NaN
7,None,O. Horiot,"""Métisse-Les Riceys"", Brut Nature",BRUT NATURE,SPARKLING,Champagne,FRA,None,550,NaN
8,None,Domaine Benoit Courault,"""Gilbourg"", VDF",CHENIN BLANC,WHITE,Loire Valley,FRA,2020,180,NaN
9,None,Weingut J. J. Prüm,"""Graacher Himmelreich"", Kabinett",RIESLING,WHITE,Mosel,DEU,2020,254,NaN


In [42]:
import requests
from bs4 import BeautifulSoup
import re

# Define the search now that we know the names
def vivino_search(name, producer, type,	region,	country, vintage, menu_price):

    # Define the base URL
    base_url = "https://www.vivino.com/search/wines"

    # Create the search query
    query = f"{name} {producer} {type} {vintage} {region} {country}"

    # Send request to Vivino search page
    params = {"q": query}
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:66.0) Gecko/20100101 Firefox/66.0"
    }

    # Send GET request
    response = requests.get(base_url, params=params, headers=headers)

    # Check if request was successful
    if response.status_code != 200:
        print("Failed to fetch data")
        return None

    # Parse the HTML response
    soup = BeautifulSoup(response.text, "html.parser")

    # Find first wine result
    first_result = soup.select_one(".card.card-lg")
    if not first_result:
        print("No results found.")
        return None

    # Extract wine details
    try:
        wine_name = first_result.select_one(".wine-card__name").text.strip()
        link = "https://www.vivino.com" + first_result.select_one("a")["href"]
        country = first_result.select_one(".wine-card__region [data-item-type='country']").text.strip()
        region = first_result.select_one(".wine-card__region .link-color-alt-grey").text.strip()
        rating = first_result.select_one(".average__number").text.strip() if first_result.select_one(".average__number") else "N/A"
        num_ratings = first_result.select_one(".text-micro").text.split(' ratings')[0].strip() if first_result.select_one(".text-micro") else "N/A"
        price = first_result.select_one(".wine-price-value").text.strip() if first_result.select_one(".wine-price-value") else "N/A"

    except AttributeError:
        print("Error extracting data")
        return None
    

    print("Result found:", wine_name)

    print("Checking link:", link)

    link_response = requests.get(link, headers=headers)
    if link_response.status_code != 200:
        print("Failed to fetch data")
        return None
    link_soup = BeautifulSoup(link_response.text, "html.parser")
    
    # Save the data as link.txt
    with open("link.txt", "w") as f:
        f.write(str(link_soup))

    try:
        food_container = link_soup.select_one(".foodPairing__foodContainer--1bvxM")

        # Extract food pairing names
        food_pairings = [str(a).split('aria-label="')[1].split('"')[0] for a in food_container.find_all("a")]

    except AttributeError:
        print("Error extracting food pairings")
        food_pairings = []

    # Extract price if not available
    if len(price) <= 1:
        # print("Price not available. Extracting from page")
        try:
            script_tag = link_soup.find("script", {"type": "application/ld+json"})

            # Load the JSON data
            json_data = json.loads(script_tag.string)

            # Extract the price
            price = json_data.get("offers", {}).get("price")
            
            if price is None:
                # Find the price element
                price_element = link_soup.find("span", class_="purchaseAvailabilityPPC__amount--2_4GT")

                # Extract the text and clean it
                price = price_element.text.strip() if price_element else "N/A"

        except AttributeError:
            print("Error extracting price")

    # Check if price is a number
    if price != "N/A" and price != "-":
        price = price.replace("$", "")
        price = price.replace(" ", "")
        price = price.replace("€", "")
        price = price.replace("£", "")
        price = price.replace("¥", "")
        price = price.replace("₩", "")
        price = price.replace("₹", "")

        try:
            price = float(price)
            price_multiplier = menu_price / price
        except ValueError:
            print("Error converting price to float")
            price_multiplier = "N/A"
    else:
        price_multiplier = "N/A"

    # Create output
    # Return wine data
    data = {
        "name": wine_name,
        "link": link,
        "country": country,
        "region": region,
        "rating": rating,
        "num_ratings": num_ratings,
        "price": price,
        "price_multiplier": price_multiplier,
        "food_pairings": food_pairings
    }

    return data


In [20]:
import requests
from bs4 import BeautifulSoup
import re

# Define the search now that we know the names
def vivino_search_mp(wine_tuple):
    name, producer, type, region, country, vintage, menu_price = wine_tuple

    # Define the base URL
    base_url = "https://www.vivino.com/search/wines"

    # Create the search query
    query = f"{name} {producer} {type} {vintage} {region} {country}"

    # Send request to Vivino search page
    params = {"q": query}
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:66.0) Gecko/20100101 Firefox/66.0"
    }

    # Send GET request
    response = requests.get(base_url, params=params, headers=headers)

    # Check if request was successful
    if response.status_code != 200:
        print("Failed to fetch data")
        return None

    # Parse the HTML response
    soup = BeautifulSoup(response.text, "html.parser")

    # Find first wine result
    first_result = soup.select_one(".card.card-lg")
    if not first_result:
        print("No results found.")
        return None

    # Extract wine details
    try:
        wine_name = first_result.select_one(".wine-card__name").text.strip()
        link = "https://www.vivino.com" + first_result.select_one("a")["href"]
        country = first_result.select_one(".wine-card__region [data-item-type='country']").text.strip()
        region = first_result.select_one(".wine-card__region .link-color-alt-grey").text.strip()
        rating = first_result.select_one(".average__number").text.strip() if first_result.select_one(".average__number") else "N/A"
        num_ratings = first_result.select_one(".text-micro").text.split(' ratings')[0].strip() if first_result.select_one(".text-micro") else "N/A"
        price = first_result.select_one(".wine-price-value").text.strip() if first_result.select_one(".wine-price-value") else "N/A"

    except AttributeError:
        print("Error extracting data")
        return None
    
    # Check if price is a number
    if price != "N/A":
        price = float(price)
        price_multiplier = menu_price / price

    # Return wine data
    data = {
        "name": wine_name,
        "link": link,
        "country": country,
        "region": region,
        "rating": rating,
        "num_ratings": num_ratings,
        "price": price,
        "price_multiplier": price_multiplier
    }

    print("Result found:", wine_name)

    print("Checking link:", link)

    link_response = requests.get(link, headers=headers)
    if link_response.status_code != 200:
        print("Failed to fetch data")
        return None
    link_soup = BeautifulSoup(link_response.text, "html.parser")
    
    # Save the data as link.txt
    with open("link.txt", "w") as f:
        f.write(str(link_soup))

    try:
        food_container = link_soup.select_one(".foodPairing__foodContainer--1bvxM")

        # Extract food pairing names
        food_pairings = [str(a).split('aria-label="')[1].split('"')[0] for a in food_container.find_all("a")]

        data["food_pairings"] = food_pairings

    except AttributeError:
        print("Error extracting food pairings")

    # Extract price if not available
    if len(price) <= 1:
        # print("Price not available. Extracting from page")
        try:
            script_tag = link_soup.find("script", {"type": "application/ld+json"})

            # Load the JSON data
            json_data = json.loads(script_tag.string)

            # Extract the price
            price = json_data.get("offers", {}).get("price")
            
            if price is None:
                # Find the price element
                price_element = link_soup.find("span", class_="purchaseAvailabilityPPC__amount--2_4GT")

                # Extract the text and clean it
                price = price_element.text.strip() if price_element else "N/A"


            data["price"] = price

        except AttributeError:
            print("Error extracting price")

    return data


In [51]:
# Test the function

# Select random row from the dataframe
row = df.sample(1).iloc[0]

print(row)

# Get wine data
wine_data = vivino_search(row["name"], row["producer"], row["type"], row["region"], row["country"], row["vintage"])

wine_data

id                              None
producer     Azienda Agricola Monban
name                  Questo Neanche
type                       COL FONDO
main_type                  SPARKLING
region                        Veneto
country                          ITA
vintage                         2021
price                             45
size                          bottle
Name: 5, dtype: object
Result found: Inama Azienda Agricola Vulcaia Fume Sauvignon del Veneto
Checking link: https://www.vivino.com/US/en/wines/1912147
Error extracting taste data


{'name': 'Inama Azienda Agricola Vulcaia Fume Sauvignon del Veneto',
 'link': 'https://www.vivino.com/US/en/wines/1912147',
 'country': 'Italy',
 'region': 'Veneto',
 'rating': '4.2',
 'num_ratings': '5051',
 'price': '$48.99',
 'food_pairings': ['Shellfish',
  'Pasta',
  'Mature and hard cheese',
  'Lean fish',
  'Cured Meat']}

In [25]:
df = pd.read_csv("menus/rake-wine.csv")

df.head()

,id,producer,name,type,main_type,region,country,vintage,price,size
0,NaN,Chëpìka,Petnat Catawba,PETNAT,WHITE,Finger Lakes,USA,2021,64.0,bottle
1,NaN,La Ferme du Vert,L' Angelou Blanc Bulle,NaN,WHITE,Southwest,FRA,2021,56.0,bottle
2,NaN,Guiborat,Prisme - Blanc de Blancs,"BLANC DE BLANCS, GRAND CRU EXTRA BRUT",SPARKLING,Champagne,FRA,NaN,144.0,bottle
3,NaN,Pierre Moncuit,Delos Blanc de Blancs,"BLANC DE BLANCS, GRAND CRU BRUT",SPARKLING,Champagne,FRA,NaN,117.0,bottle
4,NaN,Robert Moncuit,Millésime Blanc de Blancs,"BLANC DE BLANCS, GRAND CRU BRUT",SPARKLING,Champagne,FRA,2013,196.0,bottle


In [43]:
from tqdm import tqdm
import time
import multiprocessing

# Get wine data for all wines in the dataframe
def vivino_search_all(df):
    # Create a copy of the dataframe
    new_df = df.copy()

    # Create lists to store the results
    food_pairings = []
    prices = []
    price_multipliers = []
    ratings = []
    num_ratings = []
    links = []

    # Set fail count to quite if 5 fails in a row
    fail_count = 0

    # Iterate over each row in the dataframe
    for index, row in tqdm(new_df.iterrows(), total=len(new_df)):
        # Get wine data
        wine_data = vivino_search(name=row["name"], producer=row["producer"], type=row["type"], region=row["region"], country=row["country"], vintage=row["vintage"], menu_price=row["price"])
        
        # Append the data to the list
        if wine_data:
            food_pairings.append(wine_data["food_pairings"])
            prices.append(wine_data["price"])
            ratings.append(wine_data["rating"])
            links.append(wine_data["link"])
            price_multipliers.append(wine_data["price_multiplier"])
            num_ratings.append(wine_data["num_ratings"])
            fail_count = 0
            
        else:
            food_pairings.append("N/A")
            prices.append("N/A")
            ratings.append("N/A")
            links.append("N/A")
            price_multipliers.append("N/A")
            num_ratings.append("N/A")
            fail_count += 1

        if fail_count >= 5:
            print("Failed 5 times in a row. Pausing for 3 minutes.")
            time.sleep(180)
            
        
        # Pause for a half second to avoid rate limiting
        time.sleep(0.51)

    # Add the lists to the dataframe
    new_df["food_pairings"] = food_pairings
    new_df["vivino_price"] = prices
    new_df["price_multiplier"] = price_multipliers
    new_df["rating"] = ratings
    new_df["link"] = links
    new_df["num_ratings"] = num_ratings
    
    # Rename the price column to menu_price
    new_df.rename(columns={"price": "menu_price"}, inplace=True)

    return new_df

# # Call the function
new_df = vivino_search_all(df)

new_df.head(27)

  0%|          | 0/166 [00:00<?, ?it/s]

Result found: Chëpìka Delaware Pet Nat
Checking link: https://www.vivino.com/US/en/wines/155265081
Error extracting food pairings


  1%|          | 1/166 [00:01<04:38,  1.69s/it]

Result found: Ferme du Vert Jérôme Galaup L'AngeLou Blanc Bulle
Checking link: https://www.vivino.com/US/en/wines/37976367


  1%|          | 2/166 [00:03<04:30,  1.65s/it]

...

Failed to fetch data
Failed 5 times in a row. Pausing for 10 seconds.


 99%|█████████▉| 165/166 [08:17<00:05,  5.99s/it]

Failed to fetch data
Failed 5 times in a row. Pausing for 10 seconds.


100%|██████████| 166/166 [08:28<00:00,  3.06s/it]


,id,producer,name,type,main_type,region,country,vintage,menu_price,size,food_pairings,vivino_price,price_multiplier,rating,link
0,NaN,Chëpìka,Petnat Catawba,PETNAT,WHITE,Finger Lakes,USA,2021,64.0,bottle,[],30.15,2.12272,3.9,https://www.vivino.com/US/en/wines/155265081
1,NaN,La Ferme du Vert,L' Angelou Blanc Bulle,NaN,WHITE,Southwest,FRA,2021,56.0,bottle,"[Shellfish, Poultry, Appetizers and snacks, Le...",21.59,2.593793,3.9,https://www.vivino.com/US/en/wines/37976367
2,NaN,Guiborat,Prisme - Blanc de Blancs,"BLANC DE BLANCS, GRAND CRU EXTRA BRUT",SPARKLING,Champagne,FRA,NaN,144.0,bottle,"[Pork, Shellfish, Rich fish (salmon, tuna etc)...",23.99,6.002501,4.0,https://www.vivino.com/US/en/wines/163391155
3,NaN,Pierre Moncuit,Delos Blanc de Blancs,"BLANC DE BLANCS, GRAND CRU BRUT",SPARKLING,Champagne,FRA,NaN,117.0,bottle,"[Pork, Shellfish, Rich fish (salmon, tuna etc)...",N/A,N/A,4.1,https://www.vivino.com/US/en/wines/3765256
4,NaN,Robert Moncuit,Millésime Blanc de Blancs,"BLANC DE BLANCS, GRAND CRU BRUT",SPARKLING,Champagne,FRA,2013,196.0,bottle,"[Pork, Shellfish, Rich fish (salmon, tuna etc)...",N/A,N/A,4.2,https://www.vivino.com/US/en/wines/165673198
5,NaN,Azienda Agricola Monban,Questo Neanche,COL FONDO,SPARKLING,Veneto,ITA,2021,45.0,bottle,"[Shellfish, Pasta, Mature and hard cheese, Lea...",48.99,0.918555,4.2,https://www.vivino.com/US/en/wines/1912147
6,NaN,Diletta Tonello,Marachelle,FRIZZANTE,WHITE,Veneto,ITA,NaN,53.0,bottle,"[Appetizers and snacks, Aperitif]",N/A,N/A,4.0,https://www.vivino.com/US/en/wines/174221101
7,NaN,Aldo Viola,Brutto,NaN,WHITE,Sicily,ITA,2022,49.0,bottle,"[Appetizers and snacks, Aperitif]",N/A,N/A,4.0,https://www.vivino.com/US/en/wines/160956141
8,NaN,Joao Pato,Ducking,PETNAT,WHITE,Bairrada,PRT,2022,61.0,bottle,"[Shellfish, Rich fish (salmon, tuna etc), Appe...",N/A,N/A,3.9,https://www.vivino.com/US/en/wines/167366463
9,NaN,Familie Bauer,Unsprung,PETNAT,WHITE,Wagram,AUT,2021,52.0,bottle,"[Pork, Shellfish, Poultry, Rich fish (salmon, ...",22.99,2.261853,4.1,https://www.vivino.com/US/en/wines/164244051


In [22]:
def vivino_search_multiprocess(df):
    """
    Process DataFrame rows in parallel using multiprocessing.

    Args:
        df (pd.DataFrame): Input DataFrame containing wine information

    Returns:
        pd.DataFrame: DataFrame with added Vivino search results
    """
    # Convert DataFrame to list of dictionaries for easier serialization
    records = df.to_dict("records")

    # Create a pool of processes
    pool = multiprocessing.Pool(processes=multiprocessing.cpu_count() - 1)

    # Run the search for each record in parallel
    results = list(tqdm(pool.imap(vivino_search_mp, records), total=len(records)))

    # Close the pool
    pool.close()
    pool.join()

    # Create lists to store the results
    food_pairings = []
    prices = []
    price_multipliers = []
    ratings = []
    links = []

    # Extract the results from the list
    for result in results:
        if result:
            food_pairings.append(result.get("food_pairings", []))
            prices.append(result.get("price", "N/A"))
            price_multipliers.append(result.get("price_multiplier", 1.0))
            ratings.append(result.get("rating", "N/A"))
            links.append(result.get("link", "N/A"))
        else:
            # Handle None results
            food_pairings.append([])
            prices.append("N/A")
            price_multipliers.append(1.0)
            ratings.append("N/A")
            links.append("N/A")

    # Create a new DataFrame with the results
    new_df = df.copy()
    new_df["food_pairings"] = food_pairings
    new_df["vivino_price"] = prices
    new_df["price_multiplier"] = price_multipliers
    new_df["rating"] = ratings
    new_df["link"] = links

    # Rename the price column to menu_price
    new_df.rename(columns={"price": "menu_price"}, inplace=True)

    return new_df

# Test the function
vivino_df = vivino_search_multiprocess(df.head(10))

vivino_df.head()

  0%|          | 0/10 [00:00<?, ?it/s]Process SpawnPoolWorker-26:
Process SpawnPoolWorker-23:
Process SpawnPoolWorker-22:
Process SpawnPoolWorker-25:
Process SpawnPoolWorker-24:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/Users/Austin/opt/anaconda3/envs/ANLY501/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/Users/Austin/opt/anaconda3/envs/ANLY501/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/Austin/opt/anaconda3/envs/ANLY501/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/Users/Austin/opt/anaconda3/envs/ANLY501/lib/python3.10/multiprocessing/pool.py", line 114, in worker
    task = get()
  File "/Users/Austin/opt/anaconda3/envs/ANLY501/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/Users/Austin/opt/anaconda3/en

KeyboardInterrupt: 

In [3]:
# Test functions imports
from functions import *
from dotenv import load_dotenv
import os
import pandas as pd
import requests
import json
import PyPDF2
import google.generativeai as genai
import re
import numpy as np

load_dotenv(dotenv_path='config.env')
google_key = os.getenv('GOOGLE_KEY')

if google_key is None:
    raise ValueError("GOOGLE_KEY not found in .env file")

parser = GeminiWineParser(google_key)

# Read the PDF
text = extract_text_from_pdf("menus/rake-wine.pdf")
pages = [text[i] for i in text.keys()]

# View the first page
pages[0]

'White\nChëpìka, Petnat Catawba 2021 Finger Lakes, NY 64\nLa Ferme du Vert, "L\' Angelou Blanc Bulle" Mauzac Blanc 2021 Southwest, FRA 56\nGuiborat, "Prisme - Blanc de Blancs", Grand Cru Extra Brut Chardonnay NV Champagne, FRA 144\nPierre Moncuit, "Delos Blanc de Blancs" Grand Cru Brut Chardonnay NV Champagne, FRA 117\nRobert Moncuit, "Millésime Blanc de Blancs", Grand Cru Brut Chardonnay 2013 Champagne, FRA 196\nAzienda Agricola Monban, "Questo Neanche", Col Fondo Glera 2021 Veneto, ITA 45\nDiletta Tonello, "Marachelle", Frizzante Durella blend NV Veneto, ITA 53\nAldo Viola, "Brutto" Catarratto 2022 Sicily, ITA 49\nJoao Pato, "Ducking" Petnat Sercialinho 2022 Bairrada, PRT 61\nFamilie Bauer, "Unsprung" Petnat Gruner + Roter V. + Riesling 2021 Wagram, AUT 52\nRosé\nAnnesanti, "Raspato", Frizzante Rosato Sangiovese + Aleatico 2020 Umbria, ITA 57\nLaherte Frères, "Les Beaudiers", Rosé de Saignée Pinot Meunier NV Champagne, FRA 166\nWeszeli, "We Love Petnat" Zweigelt 2020 Kamptal, AUT 57\

In [8]:
# Parse the first page
results = parser.parse_wine_list(pages[0])

print(json.dumps(results, indent=2))

[
  {
    "id": null,
    "producer": "Ch\u00ebp\u00ecka",
    "name": "Petnat Catawba",
    "type": "PETNAT",
    "main_type": "WHITE",
    "region": "Finger Lakes",
    "country": "USA",
    "vintage": "2021",
    "price": "64",
    "size": "bottle"
  },
  {
    "id": null,
    "producer": "La Ferme du Vert",
    "name": "L' Angelou Blanc Bulle",
    "type": null,
    "main_type": "WHITE",
    "region": "Southwest",
    "country": "FRA",
    "vintage": "2021",
    "price": "56",
    "size": "bottle"
  },
  {
    "id": null,
    "producer": "Guiborat",
    "name": "Prisme - Blanc de Blancs",
    "type": "BLANC DE BLANCS, GRAND CRU EXTRA BRUT",
    "main_type": "SPARKLING",
    "region": "Champagne",
    "country": "FRA",
    "vintage": null,
    "price": "144",
    "size": "bottle"
  },
  {
    "id": null,
    "producer": "Pierre Moncuit",
    "name": "Delos Blanc de Blancs",
    "type": "BLANC DE BLANCS, GRAND CRU BRUT",
    "main_type": "SPARKLING",
    "region": "Champagne",
    "c

In [34]:
from functions import *
import pandas as pd

# Read the csv
df = pd.read_csv("menus/rake-wine.csv")

print(df.shape)
df.head()

(166, 10)


,id,producer,name,type,main_type,region,country,vintage,price,size
0,NaN,Chëpìka,Petnat Catawba,PETNAT,WHITE,Finger Lakes,USA,2021,64.0,bottle
1,NaN,La Ferme du Vert,L' Angelou Blanc Bulle,NaN,WHITE,Southwest,FRA,2021,56.0,bottle
2,NaN,Guiborat,Prisme - Blanc de Blancs,"BLANC DE BLANCS, GRAND CRU EXTRA BRUT",SPARKLING,Champagne,FRA,NaN,144.0,bottle
3,NaN,Pierre Moncuit,Delos Blanc de Blancs,"BLANC DE BLANCS, GRAND CRU BRUT",SPARKLING,Champagne,FRA,NaN,117.0,bottle
4,NaN,Robert Moncuit,Millésime Blanc de Blancs,"BLANC DE BLANCS, GRAND CRU BRUT",SPARKLING,Champagne,FRA,2013,196.0,bottle


In [16]:
# Get vivino data
vivino_df = vivino_search_multiprocess(df)

vivino_df.head()

  0%|          | 0/166 [00:00<?, ?it/s]


PicklingError: Can't pickle <class 'pandas.core.frame.Pandas'>: attribute lookup Pandas on pandas.core.frame failed

In [13]:
smaller_df = vivino_df.head(75)

smaller_df.shape

(75, 14)

In [14]:
smaller_df.to_csv("rake-wine-75.csv", index=False)

In [46]:
new_df.to_csv("rake-wine-full.csv", index=False)